In [1]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.preprocessing import sequence
import numpy as np
from tqdm import tqdm
from IPython.display import clear_output

tf.enable_eager_execution()
tf.test.is_gpu_available()

True

In [2]:
import sys 
sys.path.append("..") 
from config import *

In [3]:
### Hyperparameter
###################
SAVE_PATH = '/home/huangzc/competition/tencent/model_ckpt/text_cnn/model.ckpt'
EMB_SIZE = 50
BATCH_SIZE = 10240
EPOCHS = 100

In [4]:
### Read data
tr_ad_df = pd.read_pickle(TRAIN_DIR+AD_PATH)
ts_ad_df = pd.read_pickle(TEST_DIR+AD_PATH)

tr_clk_list_df = pd.read_pickle(TRAIN_DIR+CLK_LIST_PATH)
ts_clk_list_df = pd.read_pickle(TEST_DIR+CLK_LIST_PATH)

tr_user_df = pd.read_pickle(TRAIN_DIR+USER_PATH)
tr_user_df = tr_user_df.groupby(['user_id']).agg({'age': 'first', 'gender': 'first'}).reset_index()

assert tr_user_df['user_id'].values.tolist() == tr_clk_list_df['user_id'].values.tolist()
grid_df = pd.concat([tr_clk_list_df, tr_user_df[['age']]], axis=1)

In [5]:
feature_columns = ['clk_list']
label_column = 'age'

grid_df[label_column] = grid_df[label_column] - 1

In [6]:
### Split into train and valid dataset
train = grid_df[feature_columns+[label_column]]
test = ts_clk_list_df[feature_columns]

train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

### x, y dataframe
x_train, y_train = train[feature_columns], train[label_column]
x_val, y_val = val[feature_columns], val[label_column]
x_test = test

720000 train examples
180000 validation examples
1000000 test examples


In [7]:
sentence_size = int(grid_df['clk_list'].map(lambda x: len(x)).quantile(0.9))
print('choose sentences max len: %d' % sentence_size)

choose sentences max len: 66


In [8]:
print("Pad sequences (samples x time)")

### feature_columns[0] means click list
x_train = sequence.pad_sequences(x_train[feature_columns[0]],
                                 maxlen=sentence_size, 
                                 padding='post', 
                                 truncating='post',
                                 dtype='int64',
                                 value=0
                                 )
x_val = sequence.pad_sequences(x_val[feature_columns[0]],
                                 maxlen=sentence_size, 
                                 padding='post', 
                                 truncating='post',
                                 dtype='int64',
                                 value=0
                                 )
x_test = sequence.pad_sequences(x_test[feature_columns[0]], 
                                maxlen=sentence_size, 
                                padding='post',
                                truncating='post',
                                dtype='int64',
                                value=0
                               )
print("x_train shape:", x_train.shape)
print("x_val shape:", x_val.shape)
print("x_test shape:", x_test.shape)

Pad sequences (samples x time)
x_train shape: (720000, 66)
x_val shape: (180000, 66)
x_test shape: (1000000, 66)


In [9]:
### Get tf dataset
def get_train_ds(x, y): 
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.shuffle(buffer_size=len(x))
    dataset = dataset.batch(BATCH_SIZE)
#     dataset = dataset.repeat(EPOCHS)
    return dataset

def get_test_ds(x, ): 
    dataset = tf.data.Dataset.from_tensor_slices(x).batch(BATCH_SIZE)
    return dataset

train_ds = get_train_ds(x_train, y_train.values)
valid_ds = get_train_ds(x_val, y_val.values)
test_ds = get_test_ds(x_test)

In [10]:
vocab_size = max(pd.concat([tr_ad_df, ts_ad_df], axis=0)['creative_id'].unique().tolist()) + 1 ### padding 0 need add 1

In [11]:
# weights = np.load('/home/huangzc/competition/tencent/data/train_preliminary/gensim_dict.npy')

In [12]:
### Construct Model
#################################
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
#         self.embedding = tf.keras.layers.Embedding(vocab_size, EMB_SIZE, weights=[weights])
        self.embedding = tf.keras.layers.Embedding(vocab_size, EMB_SIZE)

        # 词窗大小分别为3,4,5
        k1, k2, k3 = 3, 4, 5
        self.cnn1 = tf.keras.layers.Conv1D(64, k1, padding='same', strides=1, activation='relu', 
                                           kernel_regularizer=tf.keras.regularizers.l2())
        self.pool1 = tf.keras.layers.MaxPooling1D(pool_size=sentence_size-k1+1)
        self.cnn2 = tf.keras.layers.Conv1D(64, k2, padding='same', strides=1, activation='relu',
                                           kernel_regularizer=tf.keras.regularizers.l2())
        self.pool2 = tf.keras.layers.MaxPooling1D(pool_size=sentence_size-k2+1)
        self.cnn3 = tf.keras.layers.Conv1D(64, k3, padding='same', strides=1, activation='relu',
                                           kernel_regularizer=tf.keras.regularizers.l2())
        self.pool3 = tf.keras.layers.MaxPooling1D(pool_size=sentence_size-k3+1)
        # 合并三个模型的输出向量
        self.concat = tf.keras.layers.Concatenate(axis=-1)
        self.flat = tf.keras.layers.Flatten()
        self.drop = tf.keras.layers.Dropout(0.3)
        self.dense1 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, inputs):
        x = self.embedding(inputs)
        
        x1 = self.cnn1(x)
        x1 = self.pool1(x1)
        
        x2 = self.cnn2(x)
        x2 = self.pool2(x2)
        
        x3 = self.cnn3(x)
        x3 = self.pool3(x3)
        
        x = self.concat([x1, x2, x3])
        x = self.flat(x)
        return self.dense1(x)

model = MyModel()

In [13]:
# # Restore the weights
# model.load_weights(SAVE_PATH)

In [14]:
### Loss & Metric
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

valid_loss = tf.keras.metrics.Mean(name='valid_loss')
valid_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='valid_accuracy')

In [15]:
@tf.function
def train_step(features, labels):
    with tf.GradientTape() as tape:
        predictions = model(features)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [16]:
@tf.function
def valid_step(features, labels):
    predictions = model(features)
    v_loss = loss_object(labels, predictions)

    valid_loss(v_loss)
    valid_accuracy(labels, predictions)

In [ ]:
for epoch in range(EPOCHS):
  # 在下一个epoch开始时，重置评估指标
    train_loss.reset_states()
    train_accuracy.reset_states()
    valid_loss.reset_states()
    valid_accuracy.reset_states()

    CNT = 0
    for features, labels in train_ds:
#         clear_output(wait=True)
        train_step(features, labels)
        CNT += 1
        print('batch: %d, batch loss: %.4f' % (CNT, train_loss.result()))    
        
    for val_features, val_labels in valid_ds:
        valid_step(val_features, val_labels)

    template = 'Epoch {}, Loss: {:.2f}, Accuracy: {:.2f}%, Valid Loss: {:.2f}, Valid Accuracy: {:.2f}%'
    print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         valid_loss.result(),
                         valid_accuracy.result()*100))

batch: 1, batch loss: 2.2872
batch: 2, batch loss: 2.2809
batch: 3, batch loss: 2.2750
batch: 4, batch loss: 2.2690
batch: 5, batch loss: 2.2626
batch: 6, batch loss: 2.2562
batch: 7, batch loss: 2.2499
batch: 8, batch loss: 2.2437
batch: 9, batch loss: 2.2379
batch: 10, batch loss: 2.2317
batch: 11, batch loss: 2.2259
batch: 12, batch loss: 2.2200
batch: 13, batch loss: 2.2146
batch: 14, batch loss: 2.2086
batch: 15, batch loss: 2.2032
batch: 16, batch loss: 2.1975
batch: 17, batch loss: 2.1918
batch: 18, batch loss: 2.1864
batch: 19, batch loss: 2.1813
batch: 20, batch loss: 2.1762
batch: 21, batch loss: 2.1709
batch: 22, batch loss: 2.1659
batch: 23, batch loss: 2.1611
batch: 24, batch loss: 2.1565
batch: 25, batch loss: 2.1521
batch: 26, batch loss: 2.1479
batch: 27, batch loss: 2.1437
batch: 28, batch loss: 2.1406
batch: 29, batch loss: 2.1371
batch: 30, batch loss: 2.1336
batch: 31, batch loss: 2.1307
batch: 32, batch loss: 2.1279
batch: 33, batch loss: 2.1249
batch: 34, batch lo

batch: 55, batch loss: 1.0563
batch: 56, batch loss: 1.0538
batch: 57, batch loss: 1.0511
batch: 58, batch loss: 1.0484
batch: 59, batch loss: 1.0458
batch: 60, batch loss: 1.0430
batch: 61, batch loss: 1.0402
batch: 62, batch loss: 1.0374
batch: 63, batch loss: 1.0348
batch: 64, batch loss: 1.0321
batch: 65, batch loss: 1.0293
batch: 66, batch loss: 1.0266
batch: 67, batch loss: 1.0237
batch: 68, batch loss: 1.0210
batch: 69, batch loss: 1.0182
batch: 70, batch loss: 1.0155
batch: 71, batch loss: 1.0089
Epoch 4, Loss: 1.01, Accuracy: 65.46%, Valid Loss: 1.64, Valid Accuracy: 38.09%
batch: 1, batch loss: 0.7972
batch: 2, batch loss: 0.7935
batch: 3, batch loss: 0.7927
batch: 4, batch loss: 0.7929
batch: 5, batch loss: 0.7889
batch: 6, batch loss: 0.7861
batch: 7, batch loss: 0.7826
batch: 8, batch loss: 0.7793
batch: 9, batch loss: 0.7760
batch: 10, batch loss: 0.7740
batch: 11, batch loss: 0.7704
batch: 12, batch loss: 0.7669
batch: 13, batch loss: 0.7652
batch: 14, batch loss: 0.7617

batch: 35, batch loss: 0.0851
batch: 36, batch loss: 0.0850
batch: 37, batch loss: 0.0846
batch: 38, batch loss: 0.0843
batch: 39, batch loss: 0.0839
batch: 40, batch loss: 0.0835
batch: 41, batch loss: 0.0831
batch: 42, batch loss: 0.0828
batch: 43, batch loss: 0.0824
batch: 44, batch loss: 0.0820
batch: 45, batch loss: 0.0816
batch: 46, batch loss: 0.0811
batch: 47, batch loss: 0.0808
batch: 48, batch loss: 0.0804
batch: 49, batch loss: 0.0801
batch: 50, batch loss: 0.0797
batch: 51, batch loss: 0.0794
batch: 52, batch loss: 0.0791
batch: 53, batch loss: 0.0788
batch: 54, batch loss: 0.0786
batch: 55, batch loss: 0.0782
batch: 56, batch loss: 0.0779
batch: 57, batch loss: 0.0776
batch: 58, batch loss: 0.0773
batch: 59, batch loss: 0.0771
batch: 60, batch loss: 0.0768
batch: 61, batch loss: 0.0764
batch: 62, batch loss: 0.0761
batch: 63, batch loss: 0.0758
batch: 64, batch loss: 0.0755
batch: 65, batch loss: 0.0753
batch: 66, batch loss: 0.0750
batch: 67, batch loss: 0.0747
batch: 68,

batch: 15, batch loss: 0.0138
batch: 16, batch loss: 0.0138
batch: 17, batch loss: 0.0137
batch: 18, batch loss: 0.0136
batch: 19, batch loss: 0.0136
batch: 20, batch loss: 0.0136
batch: 21, batch loss: 0.0135
batch: 22, batch loss: 0.0135
batch: 23, batch loss: 0.0134
batch: 24, batch loss: 0.0134
batch: 25, batch loss: 0.0134
batch: 26, batch loss: 0.0134
batch: 27, batch loss: 0.0133
batch: 28, batch loss: 0.0132
batch: 29, batch loss: 0.0132
batch: 30, batch loss: 0.0132
batch: 31, batch loss: 0.0131
batch: 32, batch loss: 0.0131
batch: 33, batch loss: 0.0131
batch: 34, batch loss: 0.0131
batch: 35, batch loss: 0.0130
batch: 36, batch loss: 0.0130
batch: 37, batch loss: 0.0130
batch: 38, batch loss: 0.0130
batch: 39, batch loss: 0.0130
batch: 40, batch loss: 0.0130
batch: 41, batch loss: 0.0129
batch: 42, batch loss: 0.0129
batch: 43, batch loss: 0.0128
batch: 44, batch loss: 0.0128
batch: 45, batch loss: 0.0127
batch: 46, batch loss: 0.0127
batch: 47, batch loss: 0.0126
batch: 48,

batch: 68, batch loss: 0.0057
batch: 69, batch loss: 0.0057
batch: 70, batch loss: 0.0057
batch: 71, batch loss: 0.0056
Epoch 15, Loss: 0.01, Accuracy: 99.97%, Valid Loss: 3.48, Valid Accuracy: 34.30%
batch: 1, batch loss: 0.0054
batch: 2, batch loss: 0.0053
batch: 3, batch loss: 0.0052
batch: 4, batch loss: 0.0050
batch: 5, batch loss: 0.0052
batch: 6, batch loss: 0.0052
batch: 7, batch loss: 0.0052
batch: 8, batch loss: 0.0051
batch: 9, batch loss: 0.0051
batch: 10, batch loss: 0.0051
batch: 11, batch loss: 0.0051
batch: 12, batch loss: 0.0050
batch: 13, batch loss: 0.0050
batch: 14, batch loss: 0.0051
batch: 15, batch loss: 0.0051
batch: 16, batch loss: 0.0051
batch: 17, batch loss: 0.0050
batch: 18, batch loss: 0.0050
batch: 19, batch loss: 0.0050
batch: 20, batch loss: 0.0050
batch: 21, batch loss: 0.0050
batch: 22, batch loss: 0.0050
batch: 23, batch loss: 0.0050
batch: 24, batch loss: 0.0050
batch: 25, batch loss: 0.0050
batch: 26, batch loss: 0.0050
batch: 27, batch loss: 0.004

...

batch: 48, batch loss: 0.0001
batch: 49, batch loss: 0.0001
batch: 50, batch loss: 0.0001
batch: 51, batch loss: 0.0001
batch: 52, batch loss: 0.0001
batch: 53, batch loss: 0.0001
batch: 54, batch loss: 0.0001
batch: 55, batch loss: 0.0001
batch: 56, batch loss: 0.0001
batch: 57, batch loss: 0.0001
batch: 58, batch loss: 0.0001
batch: 59, batch loss: 0.0001
batch: 60, batch loss: 0.0001
batch: 61, batch loss: 0.0001
batch: 62, batch loss: 0.0001
batch: 63, batch loss: 0.0001
batch: 64, batch loss: 0.0001
batch: 65, batch loss: 0.0001
batch: 66, batch loss: 0.0001
batch: 67, batch loss: 0.0001
batch: 68, batch loss: 0.0001
batch: 69, batch loss: 0.0001
batch: 70, batch loss: 0.0001
batch: 71, batch loss: 0.0001
Epoch 60, Loss: 0.00, Accuracy: 100.00%, Valid Loss: 5.06, Valid Accuracy: 33.62%
batch: 1, batch loss: 0.0001
batch: 2, batch loss: 0.0001
batch: 3, batch loss: 0.0001
batch: 4, batch loss: 0.0001
batch: 5, batch loss: 0.0001
batch: 6, batch loss: 0.0001
batch: 7, batch loss: 0.

batch: 28, batch loss: 0.0001
batch: 29, batch loss: 0.0001
batch: 30, batch loss: 0.0001
batch: 31, batch loss: 0.0001
batch: 32, batch loss: 0.0001
batch: 33, batch loss: 0.0001
batch: 34, batch loss: 0.0001
batch: 35, batch loss: 0.0001
batch: 36, batch loss: 0.0001
batch: 37, batch loss: 0.0001
batch: 38, batch loss: 0.0001
batch: 39, batch loss: 0.0001
batch: 40, batch loss: 0.0001
batch: 41, batch loss: 0.0001
batch: 42, batch loss: 0.0001
batch: 43, batch loss: 0.0001
batch: 44, batch loss: 0.0001
batch: 45, batch loss: 0.0001
batch: 46, batch loss: 0.0001
batch: 47, batch loss: 0.0001
batch: 48, batch loss: 0.0001
batch: 49, batch loss: 0.0001
batch: 50, batch loss: 0.0001
batch: 51, batch loss: 0.0001
batch: 52, batch loss: 0.0001
batch: 53, batch loss: 0.0001
batch: 54, batch loss: 0.0001
batch: 55, batch loss: 0.0001
batch: 56, batch loss: 0.0001
batch: 57, batch loss: 0.0001
batch: 58, batch loss: 0.0001
batch: 59, batch loss: 0.0001
batch: 60, batch loss: 0.0001
batch: 61,

batch: 8, batch loss: 0.0001
batch: 9, batch loss: 0.0001
batch: 10, batch loss: 0.0001
batch: 11, batch loss: 0.0001
batch: 12, batch loss: 0.0001
batch: 13, batch loss: 0.0001
batch: 14, batch loss: 0.0001
batch: 15, batch loss: 0.0001
batch: 16, batch loss: 0.0001
batch: 17, batch loss: 0.0001
batch: 18, batch loss: 0.0001
batch: 19, batch loss: 0.0001
batch: 20, batch loss: 0.0001
batch: 21, batch loss: 0.0001
batch: 22, batch loss: 0.0001
batch: 23, batch loss: 0.0001
batch: 24, batch loss: 0.0001
batch: 25, batch loss: 0.0001
batch: 26, batch loss: 0.0001
batch: 27, batch loss: 0.0001
batch: 28, batch loss: 0.0001
batch: 29, batch loss: 0.0001
batch: 30, batch loss: 0.0001
batch: 31, batch loss: 0.0001
batch: 32, batch loss: 0.0001
batch: 33, batch loss: 0.0001
batch: 34, batch loss: 0.0001
batch: 35, batch loss: 0.0001
batch: 36, batch loss: 0.0001
batch: 37, batch loss: 0.0001
batch: 38, batch loss: 0.0001
batch: 39, batch loss: 0.0001
batch: 40, batch loss: 0.0001
batch: 41, b

batch: 61, batch loss: 0.0001
batch: 62, batch loss: 0.0001
batch: 63, batch loss: 0.0001
batch: 64, batch loss: 0.0001
batch: 65, batch loss: 0.0001
batch: 66, batch loss: 0.0001
batch: 67, batch loss: 0.0001
batch: 68, batch loss: 0.0001
batch: 69, batch loss: 0.0001
batch: 70, batch loss: 0.0001
batch: 71, batch loss: 0.0001
Epoch 71, Loss: 0.00, Accuracy: 100.00%, Valid Loss: 5.29, Valid Accuracy: 33.54%
batch: 1, batch loss: 0.0001
batch: 2, batch loss: 0.0001
batch: 3, batch loss: 0.0001
batch: 4, batch loss: 0.0001
batch: 5, batch loss: 0.0001
batch: 6, batch loss: 0.0001
batch: 7, batch loss: 0.0001
batch: 8, batch loss: 0.0001
batch: 9, batch loss: 0.0001
batch: 10, batch loss: 0.0001
batch: 11, batch loss: 0.0001
batch: 12, batch loss: 0.0001
batch: 13, batch loss: 0.0001
batch: 14, batch loss: 0.0001
batch: 15, batch loss: 0.0001
batch: 16, batch loss: 0.0001
batch: 17, batch loss: 0.0001
batch: 18, batch loss: 0.0001
batch: 19, batch loss: 0.0001
batch: 20, batch loss: 0.00

batch: 41, batch loss: 0.0001
batch: 42, batch loss: 0.0001
batch: 43, batch loss: 0.0001
batch: 44, batch loss: 0.0001
batch: 45, batch loss: 0.0001
batch: 46, batch loss: 0.0001
batch: 47, batch loss: 0.0001
batch: 48, batch loss: 0.0001
batch: 49, batch loss: 0.0001
batch: 50, batch loss: 0.0001
batch: 51, batch loss: 0.0001
batch: 52, batch loss: 0.0001
batch: 53, batch loss: 0.0001
batch: 54, batch loss: 0.0001
batch: 55, batch loss: 0.0001
batch: 56, batch loss: 0.0001
batch: 57, batch loss: 0.0001
batch: 58, batch loss: 0.0001
batch: 59, batch loss: 0.0001
batch: 60, batch loss: 0.0001
batch: 61, batch loss: 0.0001
batch: 62, batch loss: 0.0001
batch: 63, batch loss: 0.0001
batch: 64, batch loss: 0.0001
batch: 65, batch loss: 0.0001
batch: 66, batch loss: 0.0001
batch: 67, batch loss: 0.0001
batch: 68, batch loss: 0.0001
batch: 69, batch loss: 0.0001
batch: 70, batch loss: 0.0001
batch: 71, batch loss: 0.0001
Epoch 75, Loss: 0.00, Accuracy: 100.00%, Valid Loss: 5.38, Valid Accur

In [ ]:
# 保存权重
model.save_weights(SAVE_PATH)

In [ ]:
predicted_age = []
for features in test_ds:
    predicted_age.append(model(features).numpy())

predicted_age = np.concatenate(predicted_age, axis=0)
predicted_age = np.argmax(predicted_age, axis=1)
predicted_age += 1

In [ ]:
ts_clk_list_df['predicted_age'] = predicted_age

In [ ]:
submission_path = '/home/huangzc/competition/tencent/submission/submission_age.csv'
ts_clk_list_df[['user_id', 'predicted_age']].to_csv(submission_path, index=False)